<a href="https://colab.research.google.com/github/arijitde-marvell/lstm_sample/blob/master/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow.keras as keras
import tensorflow as tf

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [0]:
import numpy as np

In [0]:
# data
data = 'rain rain go away come again another day little johny wants to play johny wants to play again in the rain'

In [0]:
# tokenize the data
tokenizer = Tokenizer()

In [0]:




tokenizer.fit_on_texts([data])

In [0]:
sequence = tokenizer.texts_to_sequences([data])[0]

In [8]:
sequence

[1, 1, 7, 8, 9, 2, 10, 11, 12, 3, 4, 5, 6, 3, 4, 5, 6, 2, 13, 14, 1]

In [9]:
len(tokenizer.word_index)

14

now for a text geberation provided one word X, next word Y has to be predicted
"X Y" - its a bigram. X is feature & Y is target variable

In [0]:



sequences=[]

for i in range(1, len(sequence)):
    #print(sequence[i-1:i+1])
    sequences.append(sequence[i-1:i+1])
    #print(sequence[i], sequence[i+1])
    

In [0]:
sequences = np.array(sequences)
#sequences # contains bigram pairs of [feature, target]

In [12]:
sequences

array([[ 1,  1],
       [ 1,  7],
       [ 7,  8],
       [ 8,  9],
       [ 9,  2],
       [ 2, 10],
       [10, 11],
       [11, 12],
       [12,  3],
       [ 3,  4],
       [ 4,  5],
       [ 5,  6],
       [ 6,  3],
       [ 3,  4],
       [ 4,  5],
       [ 5,  6],
       [ 6,  2],
       [ 2, 13],
       [13, 14],
       [14,  1]])

In [0]:
# devide X & Y
X_train, y_train = sequences[:,0], sequences[:,1]

In [14]:
X_train

array([ 1,  1,  7,  8,  9,  2, 10, 11, 12,  3,  4,  5,  6,  3,  4,  5,  6,
        2, 13, 14])

In [0]:
tokens = len(tokenizer.word_index)+1 # need to consider the 0th index

In [16]:
tokens

15

In [17]:
len(sequence)

21

In [0]:
y_train = to_categorical(y_train, num_classes=tokens)

In [0]:
#y_train

In [0]:
model = Sequential()
model.add(Embedding(input_dim=tokens, output_dim=10, input_length=1))
model.add(LSTM(32))
# but after applying softmax, each component will be in the interval {(0,1)}(0,1), 
# and the components will add up to 1, so that they can be interpreted as probabilities.
model.add(Dense(tokens, activation='softmax'))

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 10)             150       
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                5504      
_________________________________________________________________
dense_1 (Dense)              (None, 15)                495       
Total params: 6,149
Trainable params: 6,149
Non-trainable params: 0
_________________________________________________________________


In [28]:
history = model.fit(X_train, y_train, epochs=500, verbose=1)

Epoch 1/500
1/1 [==============================] - 0s 2ms/step - loss: 2.7082 - accuracy: 0.0500
Epoch 2/500
1/1 [==============================] - 0s 2ms/step - loss: 2.7073 - accuracy: 0.1500
Epoch 3/500
1/1 [==============================] - 0s 3ms/step - loss: 2.7063 - accuracy: 0.1500
Epoch 4/500
1/1 [==============================] - 0s 3ms/step - loss: 2.7053 - accuracy: 0.2000
Epoch 5/500
1/1 [==============================] - 0s 1ms/step - loss: 2.7043 - accuracy: 0.2000
Epoch 6/500
1/1 [==============================] - 0s 1ms/step - loss: 2.7033 - accuracy: 0.2500
Epoch 7/500
1/1 [==============================] - 0s 1ms/step - loss: 2.7023 - accuracy: 0.2000
Epoch 8/500
1/1 [==============================] - 0s 2ms/step - loss: 2.7014 - accuracy: 0.2000
Epoch 9/500
1/1 [==============================] - 0s 3ms/step - loss: 2.7004 - accuracy: 0.3500
Epoch 10/500
1/1 [==============================] - 0s 2ms/step - loss: 2.6994 - accuracy: 0.3500
Epoch 11/500
1/1 [===========

In [0]:
def generate_text(input_text, num_predictions=10):
    result = input_text
    for j in range(num_predictions):
        seq = tokenizer.texts_to_sequences([input_text])[0]
        #print(seq)
        pred_seq = model.predict_classes(np.array([seq]))
        #print(pred_seq)
        for word, index in tokenizer.word_index.items():
            if index == pred_seq:
                out_word = word
                #print(out_word)
                break
        input_text = ' '.join(input_text.split()[1:] + [out_word])
        #print(input_text)
        result += " " + out_word
    return result

In [33]:
generate_text('johny', 5)

'johny wants to play johny wants'